# Data Loading And Preproces

In [ ]:
!pip install datasets


In [ ]:
import tensorflow as tf
from pathlib import Path
import numpy as np
import datasets

In [ ]:
from datasets import load_dataset
#datasets.config.DOWNLOAD_TIMEOUT = 120

dataset = load_dataset("opus100", "el-en")




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/194k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/83.8M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/190k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
print(dataset["train"][1])

{'translation': {'el': '- Αν εσύ...', 'en': '- If you...'}}


In [ ]:
dataset


DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [ ]:
len(dataset['train']['translation'])

1000000

In [ ]:
greek=[item['el'] for item in dataset['train']['translation'][:20000]]# I took smaler data set because it didn't load into colab
eng=[item['en'] for item in dataset['train']['translation'][:20000]]

In [ ]:
greek=np.array(greek)
eng=np.array(eng)

In [ ]:
len(greek)

20000

In [ ]:
print(greek[1],"=",eng[1])

- Αν εσύ... = - If you...


In [ ]:
type(greek)


numpy.ndarray

In [ ]:
text_vect_en=tf.keras.layers.TextVectorization(max_tokens=1000,
                                               output_sequence_length=20,standardize='lower_and_strip_punctuation')
text_vect_en.adapt(eng)


text_vect_gr=tf.keras.layers.TextVectorization(max_tokens=1000,
                                               output_sequence_length=20,standardize='lower_and_strip_punctuation')
text_vect_gr.adapt([f'SOS {s} EOS' for s in greek])



In [ ]:
text_vect_gr(greek[1])# it seems that i should use masking

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([69, 98,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])>

In [ ]:
text_vect_en.get_vocabulary()[:5]


['', '[UNK]', np.str_('the'), np.str_('you'), np.str_('to')]

In [ ]:
text_vect_gr.get_vocabulary()[:5]

['', '[UNK]', np.str_('sos'), np.str_('eos'), np.str_('να')]

In [ ]:
text_vect_gr.vocabulary_size()

1000

In [ ]:
vocab_size_en=text_vect_en.vocabulary_size()
vocab_size_gr=text_vect_gr.vocabulary_size()

In [ ]:
X_train_encoder=tf.constant(eng[:18000],dtype=tf.string)
X_valid_encoder=tf.constant(eng[18000:],dtype=tf.string)

X_train_decoder=tf.constant([f"SOS {s} " for s in greek[:18000]],dtype=tf.string)
X_valid_decoder=tf.constant([f"SOS {s} " for s in greek[18000:]],dtype=tf.string)

Y_train=[f" {s} EOS" for s in greek[:18000]]
Y_valid=[f" {s} EOS" for s in greek[18000:]]

In [ ]:
type(X_train_decoder)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
text_vect_gr(X_train_decoder[1])

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([ 2, 69, 98,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])>

In [ ]:
text_vect_gr(Y_train[1])

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([69, 98,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])>

# Model_1

In [ ]:
encoder_embedding=tf.keras.layers.Embedding(vocab_size_en,
                                            output_dim=10,
                                            mask_zero=False)
decoder_embedding=tf.keras.layers.Embedding(vocab_size_gr,
                                            output_dim=10,
                                            mask_zero=False)

In [ ]:
encoder_inputs=tf.keras.layers.Input(shape=[],dtype=tf.string)
decoder_inputs=tf.keras.layers.Input(shape=[],dtype=tf.string)

encoder_vec=text_vect_en(encoder_inputs)
decoder_vec=text_vect_gr(decoder_inputs)

encoded_emb=encoder_embedding(encoder_vec)
decoded_emb=decoder_embedding(decoder_vec)

encoder=tf.keras.layers.LSTM(128,return_state=True,return_sequences=False)
encoder_outputs,*encoder_state=encoder(encoded_emb)



decoder=tf.keras.layers.LSTM(128,return_sequences=True)
decoder_outputs=decoder(decoded_emb,initial_state=encoder_state)

output_layer=tf.keras.layers.Dense(vocab_size_gr, activation='softmax')
Y_prob=output_layer(decoder_outputs)


In [ ]:
Checkpoint=tf.keras.callbacks.ModelCheckpoint(
    'model_1.keras',
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='max')

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.003,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Nadam(learning_rate=lr_schedule)

model=tf.keras.Model(inputs=[encoder_inputs,decoder_inputs],outputs=[Y_prob])
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
 metrics=["accuracy"])

In [ ]:
Y_train_vec=text_vect_gr(Y_train)
Y_valid_vec=text_vect_gr(Y_valid)

In [ ]:
decoder_outputs.shape

(None, 20, 128)

In [ ]:
encoder_outputs.shape

(None, 128)

In [ ]:
history_1 = model.fit((X_train_encoder, X_train_decoder), Y_train_vec,
                      validation_data=((X_valid_encoder, X_valid_decoder), Y_valid_vec),
                      epochs=100, callbacks=[Checkpoint])


Epoch 1/100
559/563 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7226 - loss: 1.7984
Epoch 1: val_accuracy improved from -inf to 0.76190, saving model to model_1.keras
563/563 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.7228 - loss: 1.7954 - val_accuracy: 0.7619 - val_loss: 1.3910
Epoch 2/100
562/563 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7837 - loss: 1.2598
Epoch 2: val_accuracy improved from 0.76190 to 0.77125, saving model to model_1.keras
563/563 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.7837 - loss: 1.2597 - val_accuracy: 0.7712 - val_loss: 1.2997
Epoch 3/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7879 - loss: 1.1965
Epoch 3: val_accuracy improved from 0.77125 to 0.77588, saving model to model_1.keras
563/563 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.7879 - loss: 1.1965 - val_accuracy: 0.7759 - val_loss: 1.2441
Epoch 4/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7905 - loss: 1.1456
Epoch 4: val_accuracy improved from 0.7

In [ ]:
def translate(sentence_en):
  translation = ""
  for word_idx in range(20):
    X=[sentence_en]
    X_dec = ["startofseq " + translation]  # decoder input
    y_proba = model.predict((X, X_dec))[0, word_idx]
    predicted_word_id = np.argmax(y_proba)
    predicted_word = text_vect_gr.get_vocabulary()[predicted_word_id]
    if predicted_word == "endofseq":
      break
    translation += " " + predicted_word
  return translation.strip()


In [ ]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(20):
        X = tf.constant(sentence_en)  # 1D list
        X_dec = ["startofseq " + translation]  # 1D list

        y_proba = model.predict([X, X_dec])[0, word_idx]  # pass as list of inputs
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vect_gr.get_vocabulary()[predicted_word_id]

        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word

    return translation.strip()


In [ ]:
translate('Hi world')

ValueError: Unrecognized data type: x=[<tf.Tensor: shape=(), dtype=string, numpy=b'Hi world'>, ['startofseq ']] (of type <class 'list'>)